# Demonstração - Algoritmo de Classificação (Regressão Logística)

Neste tutorial, usaremos o conjunto de dados do Titanic para prever quais passageiros sobreviveriam ao acidente. O conjunto de dados inclui informações sobre cada passageiro, a cabine em que ficou, o gênero entre outros.

Etapas:
1. Preparação de dados;
1. Escolha o algoritmo (vamos usar a regressão logística);
1. **Opcional:** Ajuste de hiperparâmetros (não será feito neste tutorial);
1. Treinar o modelo;
1. Avaliar sua performance.

## Preparação de dados

1. Baixar e dividir dados;
1. Adicionar e remover colunas;
1. Lidar com valores nulos;
1. Mudar a escala numérica;
1. Mudar o tipo de dado e/ou criar categorias;
1. Selecionar as variáveis de interesse.




### Importando os dados

Precisamos carregar os dados na memória baixando-os de um site, banco de dados, data warehouse, ferramenta SaaS, etc. Uma vez baixados, podemos carregá-los na memória para operar rapidamente. Antes de dividirmos os dados, precisaremos determinar qual coluna queremos prever. Neste tutorial, vamos prever quais passageiros sobreviveriam.

In [ ]:
import pandas as pd

# Importando os dados
df = pd.read_csv('/content/titanic_survival.csv') 
label_feature_name = 'Survived'

X = df.drop(columns=[label_feature_name]) #definição das variáveis explicativas
y = df[label_feature_name] #definição da variável target

### Divisão dados

Vamos dividir os dados em 2 partes: uma para treinar o modelo (também conhecido como conjunto de treinamento) e uma para avaliar o desempenho do modelo (também conhecido como conjunto de teste). 

O conjunto de treino, neste caso, terá 80% das linhas dos dados originais. Existem diferentes estratégias para dividir os dados, no entanto, um método comum é estratificar os dados para que haja um número representativo de linhas no conjunto de treinamento e no conjunto de teste. 


In [ ]:
from sklearn.model_selection import train_test_split 

X_train_raw, X_test_raw, y_train, y_test = train_test_split( X, y,
                                                            stratify=y,
                                                            test_size=0.2)

### Adicionar colunas

Os dados que baixamos podem não ter todas as colunas que precisamos. Podemos querer adicionar mais algumas colunas combinando colunas existentes ou realizando algum tipo de cálculo. Por exemplo, você pode querer criar uma coluna chamada “year” que extrai o ano de um valor de data da coluna de aniversário.


In [ ]:
df = X_train_raw.copy()

# Adicione uma coluna para determinar se a pessoa pode votar
df['can_vote'] = df['Age'].apply(lambda age: 1 if age >= 18 else 0)

# 892 passageiros podem votar, ou seja, eles tem 18 anos ou são mais velhos
df['can_vote'].value_counts()

# Letra de cabine: uma cabine pode ser identificada como B123, então a letra da cabine será B.
df.loc[:, 'cabin_letter'] = df['Cabin'].apply(
    lambda cabin: cabin[0] if cabin and type(cabin) is str else None)

### Remover colunas

Pode haver colunas com as quais você acha que o modelo não deveria aprender. Por exemplo, o modelo pode não se importar com IDs de usuário ou endereços de e-mail específicos (o domínio de e-mail pode ser importante). Nesses casos, queremos remover essas colunas dos dados. 

Ao remover essas colunas, ajudamos o modelo a se "concentrar" no que importa, em vez de tentar entender os dados que não têm impacto na previsão. Por exemplo, a identificação de um passageiro provavelmente tem muito pouco impacto sobre se ele sobreviveu ao naufrágio do Titanic.


In [ ]:
df = df.drop(columns=['Name', 'PassengerId'])

# Name e PassengerId não são mais uma coluna do dataset
df.columns.tolist()

['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked',
 'can_vote',
 'cabin_letter']

### Imputar valores

Seus dados podem ter valores ausentes em uma coluna específica. O modelo tem dificuldade em saber o que fazer com valores ausentes. Podemos ajudá-lo preenchendo esses valores ausentes usando alguma heurística. Por exemplo, há muitos valores ausentes na coluna "Cabine". Para aqueles sem cabine conhecida, preencheremos o valor “em algum lugar fora de vista”. Para aqueles com idade ausente, usaremos a idade mediana para preencher esses valores ausentes.


In [ ]:
from sklearn.impute import SimpleImputer

print(f'Missing values in "Cabin": {len(df[df["Cabin"].isna()].index)}')
df.loc[df['Cabin'].isna(), 'Cabin'] = 'Fora de vista'
df.loc[df['cabin_letter'].isna(), 'cabin_letter'] = 'ZZZ'

print(f'Missing values in "Age": {len(df[df["Age"].isna()].index)}')
age_imputer = SimpleImputer(strategy='median')
df.loc[:, ['Age']] = age_imputer.fit_transform(df[['Age']])

print(f'Missing values in "Embarked": {len(df[df["Embarked"].isna()].index)}')
df.loc[df['Embarked'].isna(), 'Embarked'] = 'Não ha dados'

Missing values in "Cabin": 553
Missing values in "Age": 145
Missing values in "Embarked": 2


### "Normalizado" os dados

Ajuste os valores das colunas numéricas para que fiquem dentro de intervalos semelhantes para que números grandes (como segundos) não afetem a previsão de forma desproporcional tanto quanto valores menores (como idade). Existem várias estratégias de dimensionamento, como dimensionador padrão e normalizador. Para mais informações, confira este [tópico](https://datascience.stackexchange.com/questions/45900/when-to-use-standard-scaler-and-when-normalizer).


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df.loc[:, ['Age']] = scaler.fit_transform(df[['Age']])

#normalizar balance

### Codificar valores

Os algoritmos realizam operações matemáticas usando números. Devemos converter colunas que contêm strings em uma representação numérica. Uma técnica comum é codificar valores categóricos. Por exemplo, podemos converter o valor “male” para 0 e “female” para 1. 

**Nota:** para fazer isso, usaremos `OneHotEncoder()` fornecido pelo sklearn. Basicamente, ele transformará uma coluna categórica disso:

| estado cívil 
|----------|
| single   | 
| divorced | 
| married  | 

...em algo assim...

| marital_single | marital_divorced | marital_married | 
|----------------|------------------|-----------------|
| 1              | 0                | 0               | 
| 0              | 1                | 0               | 
| 0              | 0                | 1               | 


In [ ]:
from sklearn.preprocessing import OneHotEncoder

categorical_columns = ['Pclass', 'Sex', 'Embarked', 'cabin_letter']
categorical_encoder = OneHotEncoder(handle_unknown='ignore')
categorical_encoder.fit(df[categorical_columns])

# Adicione as novas colunas aos dados
new_column_names = []
for idx, cat_column_name in enumerate(categorical_columns):
    values = categorical_encoder.categories_[idx]
    new_column_names += [f'{cat_column_name}_{value}' for value in values]

df.loc[:, new_column_names] = \
  categorical_encoder.transform(df[categorical_columns]).toarray()

### Selecionar as variávies

Agora que preparamos nossos dados, precisamos selecionar as variáveis com os quais queremos que nosso modelo aprenda. Existem muitas técnicas para fazer isso. Para este tutorial, simplesmente selecionaremos os recursos (variáveis) que adicionamos, dimensionamos ou codificamos manualmente.



In [ ]:
features_to_use = [
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'can_vote',
] + new_column_names
X_train = df[features_to_use].copy()

## Escolhendo o algoritmo

Uma vez que nossos dados estejam em um estado pronto para serem treinados, devemos escolher um algoritmo para usar. Diferentes algoritmos são mais adequados para diferentes tipos de problemas e diferentes tipos de dados. Para este tutorial, usaremos um algoritmo básico chamado *Regressão Logística* que nos ajudará a classificar quais passageiros sobreviveram ao acidente do Titanic.


In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=10000)

## Treinamento do modelo

Pegamos os dados que foram preparados (X_train) e os resultados reais (y_train) para cada linha (ou seja, se o passageiro sobreviveu ao Titanic) e os alimentamos ao modelo. O modelo aprenderá observando os valores em cada coluna e vendo qual resultado ela produz (1 para sobreviveu, 0 para não sobreviveu). Depois que o modelo aprender com todos os dados, ele terminará o treinamento e poderá ser usado para fazer previsões sobre dados não vistos.


In [ ]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

## Avaliação da performance

1. Preparar os dados de teste;
1. Usar o modelo para prever os dados de teste;
1. Calcular a acurácia do mdelo
1. Determine o desempenho da linha de base e compare



### Preparar os dados de teste

Primeiro, vamos preparar nossos dados de teste (adicionar colunas, remover colunas, imputar valores, dimensionar valores, codificar valores e selecionar recursos) da mesma forma que fizemos para nosso conjunto de treino. Uma ressalva é que não vamos “encaixar” nosso scaler padrão ou nossos codificadores porque queremos apenas “encaixar” aqueles no conjunto do treino.


In [ ]:
X_test = X_test_raw.copy()

# Add colunas
X_test['can_vote'] = X_test['Age'].apply(lambda age: 1 if age >= 18 else 0)
X_test.loc[:, 'cabin_letter'] = X_test['Cabin'].apply(
    lambda cabin: cabin[0] if cabin and type(cabin) is str else None,
)

# Remover colunas
X_test = X_test.drop(columns=['Name', 'PassengerId'])

# Imputar valores
X_test.loc[X_test['Cabin'].isna(), 'Cabin'] = 'somewhere out of sight'
X_test.loc[X_test['cabin_letter'].isna(), 'cabin_letter'] = 'ZZZ'
X_test.loc[:, ['Age']] = age_imputer.transform(X_test[['Age']])
X_test.loc[X_test['Embarked'].isna(), 'Embarked'] = 'no idea'

# Normalizar colunas
X_test.loc[:, ['Age']] = scaler.transform(X_test[['Age']])

# Recodificar valores
X_test.loc[:, new_column_names] = categorical_encoder.transform(
    X_test[categorical_columns],
).toarray()

# Selecionar as variáveis
X_test = X_test[features_to_use].copy()

### Usar o modelo para prever dados de teste

Em seguida, usamos o modelo para prever quem sobrevive a partir dos dados de teste (lembre-se de que dividimos os dados anteriormente durante a preparação dos dados).


In [ ]:
y_pred = classifier.predict(X_test)

### Calcular a acurácia do modelo

Os modelos de regressão e classificação possuem métricas diferentes que são usadas para avaliar o desempenho do modelo. Como estamos usando um modelo de classificação (apesar de ser chamado de regressão logística, ele pode ser usado para classificar), usaremos a acurácia como nossa métrica. Também poderíamos usar a precisão e recall como métricas, por exemplo.


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy score: {accuracy}')

Accuracy score: 0.7541899441340782


### Determine o desempenho da linha de base e comparar

Para que possamos entender o quão boa é essa acurácia, precisamos estabelecer uma linha de base. Neste exemplo específico, a precisão da linha de base será o número de pessoas que não sobreviveram no conjunto de teste dividido pelo número de linhas no conjunto de teste.


In [ ]:
baseline_accuracy_score = y_test.value_counts()[0] / len(y_test)

print(f'Model performance.  : {accuracy}')
print(f'Baseline performance: {baseline_accuracy_score}')

Model performance.  : 0.7541899441340782
Baseline performance: 0.6145251396648045
